In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "ab080-batch-events"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='83aeec42-3e63-4652-aff5-506252e7cb67', created_at=datetime.datetime(2025, 7, 27, 5, 56, 44, 321516, tzinfo=TzInfo(UTC)), name='776014ce-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 5, 56, 44, 321516, tzinfo=TzInfo(UTC)))


In [6]:
# Test non streaming response
from typing import cast
from agentex.types import Event
from agentex.types.agent_rpc_params import ParamsSendEventRequest

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

concurrent_event_messages: list[ParamsSendEventRequest] = [
    {
        "content": {"type": "text", "author": "user", "content": "Hello, what can you do?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Can you tell me a joke?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "What is the capital of France?"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Write a short story about a cat"},
        "task_id": task.id,
    },
    {
        "content": {"type": "text", "author": "user", "content": "Tell me how an LLM works"},
        "task_id": task.id,
    },
]

events: list[Event] = []

for event_message in concurrent_event_messages:
    rpc_response = client.agents.rpc_by_name(
        agent_name=AGENT_NAME,
        method="event/send",
        params=event_message
    )

    event = cast(Event, rpc_response.result)
    events.append(event)

for event in events:
    print(event)

Event(id='f83319aa-6f63-4495-be1e-6ca52595e865', agent_id='406f9f42-9f3f-4bb2-869c-d6b36028e487', sequence_id=225, task_id='83aeec42-3e63-4652-aff5-506252e7cb67', content=TextContent(author='user', content='Hello, what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 6, 0, 8, 557860, tzinfo=TzInfo(UTC)))
Event(id='be4b68b3-991a-4dfe-8be5-31f1cc032ef1', agent_id='406f9f42-9f3f-4bb2-869c-d6b36028e487', sequence_id=226, task_id='83aeec42-3e63-4652-aff5-506252e7cb67', content=TextContent(author='user', content='Can you tell me a joke?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 6, 0, 8, 615897, tzinfo=TzInfo(UTC)))
Event(id='3001a88f-de80-44bf-b2ff-686428da043b', agent_id='406f9f42-9f3f-4bb2-869c-d6b36028e487', sequence_id=227, task_id='83aeec42-3e63-4652-aff5-506252e7cb67', content=TextContent(author='user', content='What is the capital of France?', attach

In [7]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭──────────────────────── AGENT [07/27/2025 06:00:18] ─────────────────────────╮
│ Processed event IDs: ['f83319aa-6f63-4495-be1e-6ca52595e865',                │
│ 'be4b68b3-991a-4dfe-8be5-31f1cc032ef1']                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:00:33] ─────────────────────────╮
│ Processed event IDs: ['3001a88f-de80-44bf-b2ff-686428da043b',                │
│ '49ca99da-1c88-41f3-abea-91845cd927d5',                                      │
│ 'da567964-5fa6-419d-a573-66e673f18669']                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages
